In [8]:
import streamlit as st
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from io import StringIO
import time

# --- Configuração da Página ---
st.set_page_config(
    page_title="Dashboard de Análise de Fluxo",
    page_icon="📊",
    layout="wide",
    initial_sidebar_state="expanded"
)

# --- Funções de Cálculo (O motor do aplicativo) ---

def processar_dados(texto_dados):
    """Lê o texto colado, limpa e prepara os dados para análise."""
    if not texto_dados:
        return pd.DataFrame()
    try:
        # Usar StringIO para ler o texto como se fosse um arquivo
        # O separador '\s\s+' tenta lidar com múltiplos espaços ou tabulações
        dados = pd.read_csv(StringIO(texto_dados), sep='\s\s+', engine='python', header=0)

        # Renomear colunas para garantir consistência
        dados.columns = ['Data', 'Compradora', 'Valor', 'Quantidade', 'Vendedora', 'Agressor']

        # Limpeza e conversão de tipos
        dados['Data'] = pd.to_datetime(dados['Data'], format='%d/%m/%Y%H:%M:%S', errors='coerce')
        dados['Valor'] = pd.to_numeric(dados['Valor'].str.replace('.', '').str.replace(',', '.'), errors='coerce')
        dados['Quantidade'] = pd.to_numeric(dados['Quantidade'], errors='coerce')

        # Remover linhas com dados inválidos que não puderam ser convertidos
        dados.dropna(inplace=True)

        return dados
    except Exception as e:
        st.error(f"Erro ao processar os dados: {e}. Verifique o formato do texto colado.")
        return pd.DataFrame()

def calcular_metricas(df):
    """Calcula todas as métricas necessárias para os gráficos."""
    # Lógica de Agressão e Passividade
    df['Agg_Compradora'] = df.apply(lambda row: row['Quantidade'] if row['Agressor'] == 'Comprador' else 0, axis=1)
    df['Agg_Vendedora'] = df.apply(lambda row: row['Quantidade'] if row['Agressor'] == 'Vendedor' else 0, axis=1)
    df['Pass_Compradora'] = df.apply(lambda row: row['Quantidade'] if row['Agressor'] == 'Vendedor' else 0, axis=1)
    df['Pass_Vendedora'] = df.apply(lambda row: row['Quantidade'] if row['Agressor'] == 'Comprador' else 0, axis=1)

    return df

# --- Estado da Sessão (Para guardar informações entre interações) ---
if 'dados_processados' not in st.session_state:
    st.session_state.dados_processados = pd.DataFrame()
if 'dados_carregados' not in st.session_state:
    st.session_state.dados_carregados = False
if 'playback_index' not in st.session_state:
    st.session_state.playback_index = 0
if 'is_playing' not in st.session_state:
    st.session_state.is_playing = False

# --- Interface do Usuário (O que você vê na tela) ---

st.title("📊 Dashboard de Análise de Fluxo de Ordens")
st.markdown("---")

# --- Abas de Navegação ---
tab_dados, tab_evolucao, tab_raiox, tab_regiao, tab_micro = st.tabs([
    "1. Dados",
    "2. Evolução no Tempo",
    "3. Raio-X do Fluxo",
    "4. Atuação por Região",
    "5. Microestrutura"
])

# --- Aba 1: Dados ---
with tab_dados:
    st.header("Carregar Dados do Times and Trades")
    texto_dados_input = st.text_area(
        "Cole aqui os dados do Times and Trades para carregar na base de dados",
        height=300,
        key="dados_input"
    )

    if st.button("Processar e Carregar"):
        with st.spinner("Processando..."):
            dados_brutos = processar_dados(texto_dados_input)
            if not dados_brutos.empty:
                st.session_state.dados_processados = calcular_metricas(dados_brutos)
                st.session_state.dados_carregados = True
                st.session_state.playback_index = len(st.session_state.dados_processados) -1
                st.success(f"{len(dados_brutos)} trades carregados com sucesso!")
            else:
                st.session_state.dados_carregados = False
                st.warning("Nenhum dado válido foi carregado.")

# --- Barra Lateral de Controle ---
with st.sidebar:
    st.header("Filtros Globais")

    if st.session_state.dados_carregados:
        df = st.session_state.dados_processados

        # Seleção de Players
        all_players = sorted(pd.concat([df['Compradora'], df['Vendedora']]).unique())
        player_volumes = df.groupby('Compradora')['Quantidade'].sum().add(df.groupby('Vendedora')['Quantidade'].sum(), fill_value=0).sort_values(ascending=False)
        top_5_players = player_volumes.head(5).index.tolist()

        st.subheader("Seleção de Players")
        selected_players = st.multiselect(
            "Selecione os players para análise:",
            options=all_players,
            default=top_5_players
        )

        # Agrupamento de Tempo
        st.subheader("Agrupamento de Tempo")
        time_frame = st.selectbox(
            "Compilar gráficos por:",
            options=['1 segundo', '5 segundos', '15 segundos', '30 segundos', '1 minuto', '5 minutos', '15 minutos'],
            index=4 # Padrão '1 minuto'
        )
        time_map = {
            '1 segundo': '1S', '5 segundos': '5S', '15 segundos': '15S',
            '30 segundos': '30S', '1 minuto': '1T', '5 minutos': '5T', '15 minutos': '15T'
        }
        resample_rule = time_map[time_frame]

        # Controle de Timelapse
        st.subheader("Controle de Timelapse")

        # Agrupar dados pelo time frame escolhido para o slider
        time_steps = df.set_index('Data').resample(resample_rule).size().index

        if len(time_steps) > 1:
            st.session_state.playback_index = st.slider(
                "Navegue pelo tempo:",
                min_value=0,
                max_value=len(time_steps) - 1,
                value=st.session_state.playback_index,
                format="%d"
            )
            current_time_step = time_steps[st.session_state.playback_index]
            st.info(f"Visualizando dados até: {current_time_step.strftime('%H:%M:%S')}")
        else:
            current_time_step = df['Data'].max() if not df.empty else pd.Timestamp.now()


        # Botões de Playback
        col1, col2, col3 = st.columns(3)
        with col1:
            if st.button("⏪ Voltar"):
                st.session_state.playback_index = max(0, st.session_state.playback_index - 1)
        with col2:
            if st.button("Play/Pause ▶️⏸️"):
                st.session_state.is_playing = not st.session_state.is_playing
        with col3:
            if st.button("Avançar ⏩"):
                st.session_state.playback_index = min(len(time_steps) - 1, st.session_state.playback_index + 1)

        speed = st.selectbox("Velocidade:", [1, 2, 3, 4, 5], index=0)
        repeat = st.checkbox("Repetir")

    else:
        st.info("Carregue os dados na aba '1. Dados' para habilitar os filtros.")


# --- Lógica de atualização dos gráficos com base no playback ---
if st.session_state.dados_carregados:
    df_filtrado_tempo = df[df['Data'] <= current_time_step]

    # --- Aba 2: Evolução no Tempo ---
    with tab_evolucao:
        st.header("Evolução dos Players no Tempo")

        tipo_saldo = st.radio(
            "Selecione o tipo de análise:",
            options=['Saldo Resultante', 'Agressão Líquida', 'Passivo Líquido', 'Apenas Compras', 'Apenas Vendas'],
            horizontal=True
        )

        if not df_filtrado_tempo.empty and selected_players:
            # Lógica para calcular o saldo de cada player
            data_plot = []
            for player in selected_players:
                # Compras e Vendas do player
                compras = df_filtrado_tempo[df_filtrado_tempo['Compradora'] == player]['Quantidade'].sum()
                vendas = df_filtrado_tempo[df_filtrado_tempo['Vendedora'] == player]['Quantidade'].sum()

                # Agressões
                agg_compra = df_filtrado_tempo[(df_filtrado_tempo['Compradora'] == player) & (df_filtrado_tempo['Agressor'] == 'Comprador')]['Quantidade'].sum()
                agg_venda = df_filtrado_tempo[(df_filtrado_tempo['Vendedora'] == player) & (df_filtrado_tempo['Agressor'] == 'Vendedor')]['Quantidade'].sum()

                # Passivos
                pass_compra = df_filtrado_tempo[(df_filtrado_tempo['Compradora'] == player) & (df_filtrado_tempo['Agressor'] == 'Vendedor')]['Quantidade'].sum()
                pass_venda = df_filtrado_tempo[(df_filtrado_tempo['Vendedora'] == player) & (df_filtrado_tempo['Agressor'] == 'Comprador')]['Quantidade'].sum()

                saldo = 0
                if tipo_saldo == 'Saldo Resultante':
                    saldo = compras - vendas
                elif tipo_saldo == 'Agressão Líquida':
                    saldo = agg_compra - agg_venda
                elif tipo_saldo == 'Passivo Líquido':
                    saldo = pass_compra - pass_venda
                elif tipo_saldo == 'Apenas Compras':
                    saldo = compras
                elif tipo_saldo == 'Apenas Vendas':
                    saldo = vendas

                data_plot.append({'Player': player, 'Saldo': saldo})

            df_plot = pd.DataFrame(data_plot)

            col_graf, col_tab = st.columns([3, 1])
            with col_graf:
                fig = px.bar(df_plot, x='Player', y='Saldo', color='Player', title=f"Evolução até {current_time_step.strftime('%H:%M:%S')}")
                st.plotly_chart(fig, use_container_width=True)
            with col_tab:
                st.dataframe(df_plot.set_index('Player'))
        else:
            st.info("Selecione players para visualizar a evolução.")


    # --- Aba 3: Raio-X do Fluxo ---
    with tab_raiox:
        st.header("Raio-X do Fluxo: Delta e Volume")
        if not df_filtrado_tempo.empty:
            df_resampled = df_filtrado_tempo.set_index('Data').resample(resample_rule).agg({
                'Agg_Compradora': 'sum',
                'Agg_Vendedora': 'sum',
                'Quantidade': 'sum'
            }).reset_index()

            df_resampled['Delta'] = df_resampled['Agg_Compradora'] - df_resampled['Agg_Vendedora']

            fig = go.Figure()
            # Gráfico de Delta
            fig.add_trace(go.Bar(
                x=df_resampled['Data'],
                y=df_resampled['Delta'],
                name='Saldo Delta',
                marker_color=['green' if d > 0 else 'red' for d in df_resampled['Delta']],
                yaxis='y1'
            ))
            # Gráfico de Volume
            fig.add_trace(go.Bar(
                x=df_resampled['Data'],
                y=df_resampled['Quantidade'],
                name='Volume Total',
                marker_color='lightblue',
                yaxis='y2'
            ))

            fig.update_layout(
                title='Saldo Delta vs. Volume Total de Agressão',
                xaxis_title='Tempo',
                yaxis_title='Saldo Delta',
                yaxis2=dict(
                    title='Volume Total',
                    overlaying='y',
                    side='right'
                ),
                barmode='relative',
                legend=dict(x=0, y=1.1, orientation='h')
            )
            st.plotly_chart(fig, use_container_width=True)

    # --- Aba 4: Atuação por Região ---
    with tab_regiao:
        st.header("Análise de Atuação por Região")
        player_selecionado = st.selectbox(
            "Selecione um player para análise detalhada:",
            options=all_players if 'all_players' in locals() else []
        )
        if player_selecionado and not df_filtrado_tempo.empty:
            df_player = df_filtrado_tempo[
                (df_filtrado_tempo['Compradora'] == player_selecionado) |
                (df_filtrado_tempo['Vendedora'] == player_selecionado)
            ]

            vap_data = df_player.groupby('Valor').agg(
                Agg_Volume=('Agg_Compradora', lambda x: x[df_player['Compradora'] == player_selecionado].sum() + x[df_player['Vendedora'] == player_selecionado].sum()),
                Pass_Volume=('Pass_Compradora', lambda x: x[df_player['Compradora'] == player_selecionado].sum() + x[df_player['Vendedora'] == player_selecionado].sum())
            ).reset_index()

            fig = go.Figure()
            fig.add_trace(go.Bar(
                y=vap_data['Valor'],
                x=vap_data['Agg_Volume'],
                name='Volume de Agressão',
                orientation='h',
                marker_color='blue'
            ))
            fig.add_trace(go.Bar(
                y=vap_data['Valor'],
                x=vap_data['Pass_Volume'],
                name='Volume de Absorção',
                orientation='h',
                marker_color='orange'
            ))
            fig.update_layout(
                title=f'VAP de Agressão vs. Absorção para {player_selecionado}',
                barmode='stack',
                yaxis_title='Preço (Valor)',
                xaxis_title='Volume',
                yaxis=dict(autorange="reversed") # Preços mais altos em cima
            )
            st.plotly_chart(fig, use_container_width=True)

    # --- Aba 5: Microestrutura ---
    with tab_micro:
        st.header("Microestrutura: Esforço vs. Resultado")
        if not df_filtrado_tempo.empty:
            df_filtrado_tempo['Delta'] = df_filtrado_tempo['Agg_Compradora'] - df_filtrado_tempo['Agg_Vendedora']
            esforco_resultado = df_filtrado_tempo.groupby('Valor')['Delta'].sum().reset_index()

            fig = px.bar(
                esforco_resultado,
                y='Valor',
                x='Delta',
                orientation='h',
                title='Esforço (Delta) vs. Resultado (Preço)',
                labels={'Delta': 'Saldo Delta Acumulado', 'Valor': 'Nível de Preço'}
            )
            fig.update_layout(yaxis=dict(autorange="reversed"))
            st.plotly_chart(fig, use_container_width=True)


# --- Lógica de Playback Automático ---
if st.session_state.is_playing:
    max_index = len(time_steps) - 1 if 'time_steps' in locals() and len(time_steps) > 0 else 0
    if st.session_state.playback_index >= max_index:
        if repeat:
            st.session_state.playback_index = 0
        else:
            st.session_state.is_playing = False
    else:
        st.session_state.playback_index += 1

    time.sleep(1 / speed)
    st.experimental_rerun()

2025-08-03 05:03:51.266 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-03 05:03:51.268 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-03 05:03:51.269 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-03 05:03:51.270 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-03 05:03:51.271 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-03 05:03:51.275 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-03 05:03:51.277 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-03 05:03:51.278 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [9]:
%pip install streamlit

# Task
Forneça um passo a passo para um iniciante em Python sobre como executar um dashboard criado com Streamlit, incluindo como salvar o código, usar o GitHub para versionamento e implantação no Streamlit Cloud.

## Salvar o código python

### Subtask:
Copie o código do seu dashboard e salve-o em um arquivo com extensão `.py` (por exemplo, `dashboard_fluxo.py`) no seu computador.


## Criar uma conta no github

### Subtask:
Se você ainda não tem uma conta, crie uma no GitHub. O GitHub é onde você vai armazenar o código online.


## Criar um novo repositório no github

### Subtask:
No GitHub, crie um novo repositório (como se fosse uma pasta para o seu projeto). Dê um nome a ele (por exemplo, `dashboard-fluxo-streamlit`).


## Fazer upload do arquivo python para o github

### Subtask:
Dentro do seu novo repositório no GitHub, faça o upload do arquivo `dashboard_fluxo.py` que você salvou no passo 1.


## Criar uma conta no streamlit cloud

### Subtask:
Vá para o Streamlit Cloud e crie uma conta. Você pode usar sua conta do GitHub para fazer login, o que facilita a conexão.


## Implantar o aplicativo no streamlit cloud

### Subtask:
No Streamlit Cloud, você verá a opção para "New app" ou "Deploy an app". Selecione seu repositório do GitHub, o branch (geralmente `main` ou `master`) e o nome do arquivo Python principal (`dashboard_fluxo.py`).


## Aguardar a implantação

### Subtask:
O Streamlit Cloud vai ler seu código, instalar as bibliotecas necessárias e construir o aplicativo. Isso pode levar alguns minutos.


## Acessar o dashboard

### Subtask:
Assim que a implantação terminar, o Streamlit Cloud fornecerá um link público para o seu dashboard. Você poderá acessar o dashboard usando esse link em qualquer navegador.


## Summary:

### Data Analysis Key Findings

*   The process outlines the steps for a beginner to deploy a Streamlit dashboard, starting from saving the Python code locally.
*   It includes instructions for using GitHub for version control by creating an account, a new repository, and uploading the Python file.
*   The guide details creating an account on Streamlit Cloud, preferably using GitHub for easier connection.
*   A crucial step is deploying the application on Streamlit Cloud by selecting the GitHub repository, branch, and the main Python file.
*   The process highlights the waiting period required for Streamlit Cloud to build and deploy the application, including installing necessary libraries.
*   Finally, it explains how to access the deployed dashboard using the public link provided by Streamlit Cloud.

### Insights or Next Steps

*   The user now has a clear step-by-step guide to deploy their Streamlit dashboard from development to a publicly accessible link using GitHub and Streamlit Cloud.
*   A useful next step would be to guide the user on how to update their deployed dashboard by making changes to the code, pushing those changes to GitHub, and triggering a redeployment on Streamlit Cloud.


In [10]:
%pip install streamlit

# Task
Explain how to install Python libraries.

## Save the code

### Subtask:
Save the Streamlit application code to a Python file (e.g., `dashboard_fluxo.py`).


## Open a terminal

### Subtask:
Open a terminal or command prompt on your computer.


## Navigate to the file directory

### Subtask:
Use the `cd` command to go to the directory where you saved the Python file.


## Run the streamlit app

### Subtask:
Run the Streamlit app.


## Summary:

### Data Analysis Key Findings
*   The primary goal was to guide a user on how to run a Streamlit application, which involves a series of steps to be performed in the user's local environment.
*   The process began with instructions to save the provided Python code into a file named `dashboard_fluxo.py`.
*   Subsequent steps involved user actions outside of the direct control of the system, such as opening a terminal and navigating to the correct file directory using the `cd` command.
*   The final step was to provide the user with the command `streamlit run dashboard_fluxo.py` to execute in their terminal, which would start the Streamlit application server.

### Insights or Next Steps
*   Since the process relies on user actions in their local environment, it's beneficial to provide clear, platform-specific instructions (e.g., for Windows, macOS, and Linux) to minimize user confusion.


In [11]:
%pip install plotly